# Problema dos N-Corpos

Em astronomia é fácil calcular como dois corpos interagem entre si usando equações que agora são muito bem conhecidas, equações como o periodo orbital, velocidade da órbita entre outras. Porém essas equações não servem para calcular como 3 ou mais corpos vão interagir entre si devido a um aumento do caos no sistema.

Entretanto sabemos que a equação da força gravitacional que um corpo sente é:

\begin{equation}
    F = G\frac{M \, m}{r^2}
\end{equation}

Onde a massa do planeta maior é $M$, a do menor $m$ e a distância entro os dois é $r$

Essa equação serve muito bem para casos de 2 corpos em uma reta. Mas como temos mais de um corpo, precisamos agora de saber a direção dessa força, e a sua magnitude. Podemos utilizar o vetor unitário do vetor que parte do centro do corpo em termina no centro do outro para calcular a direção vetorial da força, e utilizar a magnitude do mesmo como a distância da equação. O que falta agora é ordenar com quais corpos estamos lidando por meio de numeração no subscrito:


\begin{equation}
    \vec{F_{1,2}} = G\frac{M_1 \, M_2}{r_{1,2}\,^2} \, \left(\frac{\vec{r_{1,2}}}{r_{1,2}}\right)
\end{equation}

Se levarmos em conta essa equação para um corpo que esteja numa posição onde há vários planetas ao redor, a força que está operando nele é a soma de todas as forças causadas por outro planetas:

\begin{equation}
    \vec{F_1} = G\frac{M_1 \, M_2}{r_{1,2}\,^2} \, \left(\frac{\vec{r_{1,2}}}{r_{1,2}}\right)
                \ + \ G\frac{M_1 \, M_3}{r_{1,3}\,^2} \, \left(\frac{\vec{r_{1,3}}}{r_{1,3}}\right) 
                \ + \ G\frac{M_1 \, M_4}{r_{1,4}\,^2} \, \left(\frac{\vec{r_{1,4}}}{r_{1,4}}\right) \cdots
\end{equation}

Simplificando em um somatório:
    
\begin{equation}
    \vec{F_n} = G M_n \sum_{j=1, j\neq n}^{N} \, \frac{M_j}{r_{n,j}\,^2} \, \left(\frac{\vec{r_{n,j}}}{r_{n,j}}\right)  
\end{equation}

Basta encontrar a aceleração do corpo agora:

\begin{equation}
    \vec{a_n} = \frac{\vec{F_n}}{M_n}
\end{equation}

\begin{equation}
    \vec{a_n} = G \sum_{j=1, j\neq n}^{N} \, \frac{M_j}{r_{n,j}\,^2} \, \left(\frac{\vec{r_{n,j}}}{r_{n,j}}\right)  
    \label{eq:aceleration}
\end{equation}

Agora que temos a aceleração do corpo, podemos calcular a posição e velocidade dele no futuro.

## Integração da aceleração

Apesar de termos uma equação para a aceleração, o que realmente queremos é a posição do corpo no espaço. Para isso devemos integrar numericamente a aceleração. Uma solução é que para cada frame da animação somamos a aceleração à velocidade, e a velocidade a posição, mas esse é um método bem ruim e com nenhuma precisão devido a muitos erros.

No código eu utilizo um algoritmo chamado _3 step leapfrog integration_, muito utilizada em simulações de cinemática onde se conhece a aceleração o tempo inteiro. Em sua forma mais simples, é composta de 3 passos para calcular uma posição no tempo $t+\Delta t$:

1. Calcular a posição $x$ no momento $t+\frac{\Delta t}{2}$ com a equação:

\begin{equation}
    x(t+\frac{\Delta t}{2}) \ =\  x(t) \, + \, v(t)* \frac{h}{2}
\end{equation}

2. Calcular a aceleração na posição calculada, e então atualizar a velocidade:

\begin{equation}
    v(t+\Delta t) \ =\ v(t) \, + \, a(t+\frac{\Delta}{2})* h
\end{equation}

3. Calcular a posição final utilizando a velocidade calculada acima

\begin{equation}
    x(t+\Delta t) \ =\ x(t+\frac{\Delta}{2}) \, + \, v(t+\Delta)* \frac{h}{2}
\end{equation} 

O $h$ é o passo de tempo ou _timestep_ que o algoritmo usa para calcular o próximo frame. Existem também outros métodos, como o _7 step leapfrog integration_, mas não vou me aprofundar neles.

<br> Tendo as equações, e os algorítmos podemos agora programar.

## Classe de vetor

Criamos um objeto de vetor 2d que por meio de _operator overloading_ pode ser somado, multiplicado, subtraído entre outros sem nos preucupar

In [1]:
from math import *

class v2d:
    """ Vetor 2d: v2d(x,y)"""
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __getitem__(self, i):
        if i == 0: return self.x
        elif i == 1: return self.y
        elif i != 0 and i != 1:
            raise IndexError('v2d index out of range')

    def __add__(self, _b):
        return v2d(self.x + _b.x, self.y + _b.y)

    def __iadd__(self, _b):
        return v2d(self.x + _b.x, self.y + _b.y)

    def __sub__(self, _b):
        return v2d(self.x - _b.x, self.y - _b.y)

    def __mul__(self, knst):
        return v2d(self.x * knst, self.y * knst)

    def __rmul__(self, knst):
        return self.__mul__(knst)

    def __truediv__(self, knst):
        return v2d(self.x / knst, self.y / knst)

    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)

    def __neg__(self):
        return v2d(-self.x, -self.y)

    def __repr__(self):
        return "(%.2f, %.2f)" %(self.x, self.y)

    def uVect(self):
        """Retorna vetor unitário"""
        absolute = self.__abs__()
        unitx = self.x / absolute
        unity = self.y / absolute
        return v2d(unitx, unity)

Com essa classe em mãos, podemos fazer todas as operações de formas simples:

In [2]:
a = v2d(1, 5)
b = v2d(4, -2)
a + b

(5.00, 3.00)

In [3]:
a * abs(b)

(4.47, 22.36)

In [4]:
print(a.uVect())
print(abs(a.uVect()))

(0.20, 0.98)
1.0


Agora nos falta inventar um jeito de armazenar dados do nosso sistema, e como criar uma classe que calcule dados por cima dele

In [5]:
terraLua = [[v2d(0, 0),  v2d(0, 0), 100, 'terra'],
            [v2d(-4, 0),  v2d(0, 3), 0.001, 'pedra1'],
            [v2d(-2, 6),  v2d(-4, 0), 0.001, 'pedra2'],
            [v2d(8, 0),  v2d(0, 2), 0.01, 'lua']]

Criei essa lista com dados de brincadeira e não devem ser levados a sério

A lista consiste de vários planetas representados por uma listas, onde o  primeiro elemento é a posição, o segundo a velocidade, o terceiro a massa do corpo e por último uma _string_ com o nome do planeta. Podemos inventar situações ou até mesmo simular o sistema solar colocando dados de forma correta nesse formato. 

Agora criamos uma classe para calcular e utilizar esses dados:

In [6]:
class Nbody:
    """Nbody(planetas, G=1, h=0.25)
    Recebe uma lista [posição, velocidade, massa, nome]
    """
    def __init__(self, planets, G=6.67408e-11, h=1/16):
        self.names  = [i[3] for i in planets]
        self.mass   = [i[2] for i in planets]
        self.vel    = [i[1] for i in planets]
        self.pos    = [i[0] for i in planets]
        self.acel   = [v2d() for i in planets]
        self.n      = len(planets)
        self.G      = G
        self.timestep  = h

    def calcAcel(self):
        for i in range(self.n):
            self.acel[i] = v2d()
            for j in range(self.n):
                if j == i:
                    continue
                r = self.pos[j] - self.pos[i]
                self.acel[i] += (self.G * self.mass[j] * r.uVect()
                                 / (abs(r)**2))

    def calcVel(self):
        for i in range(self.n):
            self.vel[i] = self.vel[i] + self.acel[i] * self.timestep

    def calcPos(self):
        for i in range(self.n):
            self.pos[i] = self.pos[i] + self.vel[i] * self.timestep / 2

    def nFrame(self):
        """Calcula o próximo instante de tempo"""
        self.calcPos()
        self.calcAcel()
        self.calcVel()
        self.calcPos()

    def printResults(self):
        """Imprime o resultado em formato de string
           nome: p:posição v:velocidade a:aceleração """
        for i in range(self.n):
            print("%s: p=%s v=%s a=%s"
                    %(self.names[i], self.pos[i], self.vel[i], self.acel[i]))

    def getResults(self):
        """retorna o resultado em formato de lista"""
        return [[i[0] for i in self.pos], [i[1] for i in self.pos]]

Vamos testar a classe acima:

In [7]:
sistema = Nbody(terraLua, G=1)
print("Iteração 0")
sistema.printResults()
sistema.nFrame()
print("\nIteração 1")
sistema.printResults()
print("\nIteração 2")
sistema.printResults()

Iteração 0
terra: p=(0.00, 0.00) v=(0.00, 0.00) a=(0.00, 0.00)
pedra1: p=(-4.00, 0.00) v=(0.00, 3.00) a=(0.00, 0.00)
pedra2: p=(-2.00, 6.00) v=(-4.00, 0.00) a=(0.00, 0.00)
lua: p=(8.00, 0.00) v=(0.00, 2.00) a=(0.00, 0.00)

Iteração 1
terra: p=(0.00, 0.00) v=(0.00, 0.00) a=(0.00, 0.00)
pedra1: p=(-3.99, 0.19) v=(0.39, 2.99) a=(6.24, -0.15)
pedra2: p=(-2.25, 6.00) v=(-3.95, -0.15) a=(0.82, -2.33)
lua: p=(8.00, 0.12) v=(-0.10, 2.00) a=(-1.56, -0.01)

Iteração 2
terra: p=(0.00, 0.00) v=(0.00, 0.00) a=(0.00, 0.00)
pedra1: p=(-3.99, 0.19) v=(0.39, 2.99) a=(6.24, -0.15)
pedra2: p=(-2.25, 6.00) v=(-3.95, -0.15) a=(0.82, -2.33)
lua: p=(8.00, 0.12) v=(-0.10, 2.00) a=(-1.56, -0.01)


Parece funcionar corretamente. Agora devemos criar um gráfico animado mostrando a posição no tempo dos corpos:

In [8]:
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

sistema = Nbody(terraLua, G=1) # "zerando" os dados

fig, ax = plt.subplots()
ln, = plt.plot([], [], 'ro',  animated=True)

def init():
    ax.set_xlim(-15, 15)
    ax.set_ylim(-10, 10)
    ax.set_aspect('equal', adjustable='box')
    return ln,

def drawFrame(i, body):
    body.nFrame()
    results = body.getResults()
    ln.set_data(results[0], results[1])
    return ln,

framerate=27
ani = FuncAnimation(fig, drawFrame, frames=3*framerate, fargs=(sistema,),
                    init_func=init, blit=True)


HTML(ani.to_html5_video())

E é essa a nossa simulação, cabe notar que quando eu estava nos primeiros testes, alguns corpos colidiam entre si e a integração fica doida quando isso acontece, propelindo o corpo a velocidades absurdas. 

Agora nos falta checar se essa simulação obedece as leis de kepler e talvez futuramente melhorar o algoritmo para simular mais corpos com eficiência